In [1]:
import torch

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random

In [3]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')

Using cache found in /diskb/houbowei/.cache/torch/hub/pytorch_fairseq_main
/diskb/houbowei/anaconda3/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/diskb/houbowei/anaconda3/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/diskb/houbowei/anaconda3/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'config': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/diskb/houbowei/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:419: UserWarning: 
'config' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 a

In [4]:
roberta.eval()

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dr

In [8]:
tokens:torch.Tensor = roberta.encode('Hello world!')

In [9]:
assert tokens.tolist() == [0, 31414, 232, 328, 2]

In [10]:
tokens.shape

torch.Size([5])

In [13]:
tokens

tensor([    0, 31414,   232,   328,     2])

In [12]:
assert roberta.decode(tokens) == 'Hello world!'

# TODO: Why the tokens' shape is five.

In [14]:
# extract features
last_layer_features:torch.Tensor = roberta.extract_features(tokens)

In [15]:
last_layer_features.shape

torch.Size([1, 5, 1024])

In [20]:
all_layers:list = roberta.extract_features(tokens, return_all_hiddens=True)

In [21]:
len(all_layers)

25

In [22]:
[i.shape for i in all_layers]

[torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024]),
 torch.Size([1, 5, 1024])]

In [25]:
# Download RoBERTa already finetuned for MNLI
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()  # disable dropout for evaluation

with torch.no_grad():
    # Encode a pair of sentences and make a prediction
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.',
                            'Roberta is not very optimized.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 0  # contradiction

    # Encode another pair of sentences
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.',
                            'Roberta is based on BERT.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 2  # entailment

Using cache found in /diskb/houbowei/.cache/torch/hub/pytorch_fairseq_main
100%|██████████| 751652118/751652118 [01:40<00:00, 7490851.28B/s] 
/diskb/houbowei/anaconda3/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/diskb/houbowei/anaconda3/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/diskb/houbowei/anaconda3/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'config': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/diskb/houbowei/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:419: UserWarning: 
'config' is validated against ConfigStore sc

In [26]:
prediction

2

In [27]:
roberta.register_classification_head('new_task', num_classes=3)

In [28]:
roberta.predict('new_task', tokens)

tensor([[-1.3362, -0.7393, -1.3481]], grad_fn=<LogSoftmaxBackward0>)